In [39]:
'''
Make the ID3 decision tree algorithm.
'''

'\nMake the ID3 decision tree algorithm.\n'

In [40]:
'''
Consider the movie dataset. Fill the dataset with the details like Movie name, Director name, Released Year and Movie type. Predict the next movie type for a particular director 
'''

'\nConsider the movie dataset. Fill the dataset with the details like Movie name, Director name, Released Year and Movie type. Predict the next movie type for a particular director \n'

In [41]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import graphviz

In [42]:
#read 'datasets/movies.txt' and create a csv file with the first row as header columns and each column seprated by a , 
#and save it as 'datasets/movies.csv'
#read the csv file and store it in a dataframe

#read 'datasets/movies.txt' and create a csv file with the first row as header columns and each column seprated by a ,
#and save it as 'datasets/movies.csv'
#read the csv file and store it in a dataframe


In [43]:
df = pd.read_csv('datasets/IMDB-Movie-Data.csv')
df.head()


,Title,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Revenue (Millions),Metascore
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,333.13,76.0
1,Prometheus,"Adventure,Mystery,Sci-Fi",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,126.46,65.0
2,Split,"Horror,Thriller",M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,138.12,62.0
3,Sing,"Animation,Comedy,Family",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,270.32,59.0
4,Suicide Squad,"Action,Adventure,Fantasy",David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,325.02,40.0


In [44]:
'''
gain functions
'''
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum(
        [(-counts[i] / np.sum(counts)) * np.log2(counts[i] / np.sum(counts))
         for i in range(len(elements))])
    return entropy


def InfoGain(data, split_attribute_name, target_name="Genre"):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    Weighted_Entropy = np.sum(
        [(counts[i] / np.sum(counts)) * entropy(data.where(
            data[split_attribute_name] == vals[i]).dropna()[target_name])
         for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain


def ID3(data, originaldata, features, target_attribute_name="Genre",
         parent_node_class=None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[
            np.argmax(np.unique(originaldata[target_attribute_name],
                                return_counts=True)[1])]
    elif len(features) == 0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data[target_attribute_name])[
            np.argmax(np.unique(data[target_attribute_name],
                                return_counts=True)[1])]
        item_values = [
            InfoGain(data, feature, target_attribute_name)
            for feature in features
        ]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature: {}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,
                          data,
                          features,
                          target_attribute_name,
                          parent_node_class)
            tree[best_feature][value] = subtree
        return tree 

In [45]:
#gini functions
def gini(x):
    elements, counts = np.unique(x, return_counts=True)
    gini = np.sum(
        [(-counts[i] / np.sum(counts)) * np.log2(counts[i] / np.sum(counts))
         for i in range(len(elements))])
    return gini


def giniIndex(data, target_name="Genre"):
    total_gini = gini(data[target_name])
    vals, counts = np.unique(data[target_name], return_counts=True)
    Weighted_gini = np.sum(
        [(counts[i] / np.sum(counts)) * gini(data.where(
            data[target_name] == vals[i]).dropna()[target_name])
         for i in range(len(vals))])
    Information_gini = total_gini - Weighted_gini
    return Information_gini


In [46]:
def giniID3(data, originaldata, features, target_attribute_name="Genre", parent_node_class=None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[
            np.argmax(np.unique(originaldata[target_attribute_name],
                                return_counts=True)[1])]
    elif len(features) == 0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]    
        item_values = [giniIndex(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        
        best_feature = features[best_feature_index]
        tree = {best_feature: {}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = giniID3(sub_data, data, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
        return(tree)

def predict(query, tree, default = 1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query, result)
            else:
                return result

def train_test_split(df):
    training_data = df.iloc[:80].reset_index(drop=True)
    testing_data = df.iloc[80:].reset_index(drop=True)
    return training_data,testing_data
    

In [47]:
def main():
    data = df
    originaldata = df
    features = np.array(data.columns)[:-1]
    tree = ID3(data, originaldata, features)
    #info gain
    print(InfoGain(data, 'Director', 'Genre'))
    print(InfoGain(data, 'Year', 'Genre'))
    print(InfoGain(data, 'Runtime (Minutes)', 'Genre'))
    print(InfoGain(data, 'Rating', 'Genre'))
    print(InfoGain(data, 'Revenue (Millions)', 'Genre'))
    print(InfoGain(data, 'Metascore', 'Genre'))
    #entropy
    print(entropy(data['play']))
    #print gini as a table
    table = pd.DataFrame(columns=['play'])
    table.loc[0] = [giniIndex(data, 'play')]
    print(table)
    #print gini as a graph
    sns.barplot(x=table.columns, y=table.loc[0])
    plt.show()
    #print tree
    print()
    print(tree)
main()

IndexError: index 0 is out of bounds for axis 0 with size 0